# Distracted Driving Detection Project

In [1]:
import os
from glob import glob
import random
import time
import tensorflow
import datetime
os.environ['KERAS_BACKEND'] = 'tensorflow'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # 3 = INFO, WARNING, and ERROR messages are not printed

from tqdm import tqdm

import numpy as np
import pandas as pd
from IPython.display import FileLink
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns 
%matplotlib inline
from IPython.display import display, Image
import matplotlib.image as mpimg
import cv2

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_files       
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.metrics import log_loss

from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.applications.vgg16 import VGG16


#import main.py
import sys
sys.path.append('./src/')
from main import *

Using TensorFlow backend.


In [37]:
num_train = 0
for x in os.listdir("./data/imgs/train/"):
    num_train += len(os.listdir('./data/imgs/train/' + x))
print(num_train)

22424


In [42]:
#random numbers rn
NUM_TRAIN = 8000
NUM_VAL = 1000
NUM_TEST = 10000


In [61]:
dataset = pd.read_csv('data/driver_imgs_list.csv')
dataset.head(5)

,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg
3,p002,c0,img_69092.jpg
4,p002,c0,img_92629.jpg


In [35]:
NUM_CLASSES = 10
IMG_W = 64
IMG_H = 64

RS = 22

In [38]:
def get_cv2_image(path, img_w, img_h, color_type=3):
    # Loading as Grayscale image
    if color_type == 1:
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    elif color_type == 3:
        img = cv2.imread(path, cv2.IMREAD_COLOR)
    # Reduce size
    img = cv2.resize(img, (img_w, img_h)) 
    return img


def load_train(img_w, img_h, color_type=3):
    Xtrain = []
    Ytrain = []
    # Loop over the training folder 
    for x in range(10):
        files = glob(os.path.join('data', 'imgs', 'train', 'c' + str(x), '*.jpg'))
        for file in files:
            img = get_cv2_image(file, img_w, img_h)
            Xtrain.append(img)
            Ytrain.append(x) # class label
    return shuffle(np.asarray(Xtrain), random_state = RS), shuffle(np.asarray(Ytrain), random_state = RS)


def load_test(img_w, img_h, color_type=3):
    Xtest = []
    # Loop over the training folder 
    files = glob(os.path.join('data', 'imgs', 'test', '*.jpg'))
    for file in files:
        img = get_cv2_image(file, img_w, img_h)
        Xtest.append(img)
    return shuffle(np.asarray(Xtest), random_state = RS)



In [41]:
Xtrain, Ytrain = load_train(IMG_W, IMG_H)
Xtest = load_test(IMG_W, IMG_H)

with open('./train_array.pkl', 'wb') as fid:
    pickle.dump((Xtrain, Ytrain), fid) #stores a tuple
with open('./test_array.pkl', 'wb') as fid:
    pickle.dump(Xtest, fid)

In [44]:
with open('./train_arrays.pkl', 'rb') as fid:
    Xtrain,Ytrain = pickle.load(fid)
with open('./test_array.pkl', 'rb') as fid:
    Xtest = pickle.load(fid)
print("Xtrain shape:", Xtrain.shape)
print("Ytrain shape:", Ytrain.shape)
print("Xtest shape:", Xtest.shape)

Xtrain shape: (22424, 64, 64, 3)
Ytrain shape: (22424,)
Xtest shape: (79726, 64, 64, 3)
